In [132]:
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep
import undetected_chromedriver as uc
from selenium_stealth import stealth
from selenium.webdriver.common.by import By

In [133]:
HOME_ADDRESS = "https://historicalnewspapers.lib.purdue.edu/?a=cl&cl=CL1&sp=PE&e=-------en-20--1--txt-txIN--------1"
options = Options()
#options.add_argument("--headless")
driver = uc.Chrome(options=options)
stealth(driver,
                languages=["en-US", "en"],
                vendor="Google Inc.",
                platform="Win32",
                webgl_vendor="Intel Inc.",
                renderer="Intel Iris OpenGL Engine",
                fix_hairline=True
        )

driver.implicitly_wait(5)
driver.get(HOME_ADDRESS)
input("Enter anything when captcha complete")
print(driver.title)

The Purdue Exponent (1889-2017) — Browse by title — Purdue University Newspapers Collection


In [134]:
start_year = 1936

In [135]:
calendar = driver.find_element(By.ID, 'datebrowserrichardtoplevelcalendar')

In [136]:
year = driver.find_element(By.XPATH, f"//h2[contains(text(), '{start_year}')]/following-sibling::div[1]")

In [137]:
first_link = year.find_element(
    By.XPATH,
    ".//ul[contains(@class,'nav') and contains(@class,'nav-pills')]//a[@class='nav-link'][1]"
)

In [138]:
first_link = first_link.get_attribute("href")

In [139]:
driver.get(first_link)

In [140]:
super_links = []

In [141]:
import json
from IPython.display import clear_output

num_years = 15
num_months = 12 * num_years
links_filename = f'/home/malinkyzubr/Desktop/HIST302/302hist/files/links{start_year}-{num_months}.json'


try:
    with open(links_filename, 'r') as f:
        super_links = json.load(f)
except:
    for month in range(num_months + 1):
        try:
            calendar_box = driver.find_element(By.ID, "datebrowserrichardmonthlevelcalendar")
            weeks = calendar_box.find_elements(By.XPATH, "./*[position() > 1]")
            days = calendar_box.find_elements(
                By.XPATH,
                ".//div[@class='datebrowserrichardmonthlevelcalendardaycell' and not(contains(@class, ' '))]"
            )

            for day in days:
                date = day.find_element(By.CSS_SELECTOR, ".hiddenwhennotsmall")
                link = day.find_element(By.CSS_SELECTOR, "li.list-group-item a")
                href = link.get_attribute("href")
                super_links.append((date.get_attribute("innerHTML"), href))
                print(f"Links Extracted: {len(super_links)}")
        except:
            pass
        next_month = driver.find_element(
            By.XPATH,
            "//a[span[text()='Next month']]"
        )
        href = next_month.get_attribute("href")
        driver.get(href)
    with open(links_filename, 'w') as f:
        json.dump(super_links, f)


In [142]:
import os

files = os.listdir("/home/malinkyzubr/Desktop/HIST302/302hist/files")
files = [file.replace(".txt", "") for file in files]

In [143]:
def text_get():
    container = driver.find_element(By.ID, "documentdisplayleftpanesectiontextcontainer")

    paragraphs = container.find_elements(
        By.CSS_SELECTOR,
        "p[dir='auto']"
    )

    text = "\n".join([p.text for p in paragraphs])

    driver.get(super_link)

    return text

In [144]:
super_links = [link for link in super_links if link[0] not in files]

In [145]:
import queue
# takes a minute to correct after internal error

retry_queue: queue.Queue[tuple[str, str]] = queue.Queue(maxsize=14)

files = os.listdir("/home/malinkyzubr/Desktop/HIST302/302hist/files")
files = [file.replace(".txt", "") for file in files]
super_links = [link for link in super_links if link[0] not in files]

start_time = time.time()
num_success = 0

while len(super_links) > 0:
    if (time.time() - start_time) / 60 > 10:
        print(f"Num successes in 10 minutes: {num_success}")
        num_success = 0
        start_time = time.time()
    if retry_queue.full():
        date, link = retry_queue.get()
    else:
        date, link = super_links.pop(0)
    driver.get(link)

    agg_text = ""
    container = driver.find_element(By.ID, "documentdisplayleftpanecontentscontainer")
    links = container.find_elements(By.CSS_SELECTOR, "a.pagetocnodelink")
    if not links:
        retry_queue.put((date, link))
    else:
        num_success += 1
        links = [link.get_attribute("href") for link in links]

        for link in links:
            driver.get(link)
            agg_text += text_get()
        with open(f"/home/malinkyzubr/Desktop/HIST302/302hist/files/{date}.txt", "w") as f:
            f.write(agg_text)
        files = os.listdir("/home/malinkyzubr/Desktop/HIST302/302hist/files")
        files = [file.replace(".txt", "") for file in files]

Num successes in 10 minutes: 11
Num successes in 10 minutes: 11
Num successes in 10 minutes: 10
Num successes in 10 minutes: 10
Num successes in 10 minutes: 12
Num successes in 10 minutes: 10
Num successes in 10 minutes: 11
Num successes in 10 minutes: 9
Num successes in 10 minutes: 9
Num successes in 10 minutes: 11
Num successes in 10 minutes: 10
Num successes in 10 minutes: 11
Num successes in 10 minutes: 10
Num successes in 10 minutes: 13
Num successes in 10 minutes: 11
Num successes in 10 minutes: 10
Num successes in 10 minutes: 11
Num successes in 10 minutes: 11
Num successes in 10 minutes: 11
Num successes in 10 minutes: 11
Num successes in 10 minutes: 11
Num successes in 10 minutes: 11
Num successes in 10 minutes: 10
Num successes in 10 minutes: 10
Num successes in 10 minutes: 11
Num successes in 10 minutes: 10
Num successes in 10 minutes: 12
Num successes in 10 minutes: 10
Num successes in 10 minutes: 10
Num successes in 10 minutes: 10
Num successes in 10 minutes: 11
Num succes

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="documentdisplayleftpanecontentscontainer"]"}
  (Session info: chrome=143.0.7499.17); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
#0 0x5a9f88d9516a <unknown>
#1 0x5a9f88812c4b <unknown>
#2 0x5a9f88864318 <unknown>
#3 0x5a9f88864551 <unknown>
#4 0x5a9f888b2764 <unknown>
#5 0x5a9f888afbee <unknown>
#6 0x5a9f88856c42 <unknown>
#7 0x5a9f888578f1 <unknown>
#8 0x5a9f88d5df09 <unknown>
#9 0x5a9f88d60e4d <unknown>
#10 0x5a9f88d46c51 <unknown>
#11 0x5a9f88d61a2b <unknown>
#12 0x5a9f88d2da20 <unknown>
#13 0x5a9f88d82a78 <unknown>
#14 0x5a9f88d82c49 <unknown>
#15 0x5a9f88d944c3 <unknown>
#16 0x79e646a9caa4 <unknown>
#17 0x79e646b29c6c <unknown>
